# OTX API DEMO (Anomalib Example)

## Customization Training API

Select a framework & import adapter modules.

We'll choose Anomalib here, and we'll import the following modules.

In [1]:
from otx.v2.adapters.torch.anomalib import Dataset, get_model
from otx.v2.adapters.torch.anomalib.engine import AnomalibEngine  # TODO: Import structure that needs to be changed
# or from otx.adapters.torch.anomalib import *

/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare Dataset & DataLoader
1. Prepare a dataset and enter path into Dataset

    - Convert to OTX's DatasetEntity and Label Schema by leveraging Datumaro's features through paths (path -> Datumaro -> OTX DatasetEntity & LabelSchema)

In [2]:
dataset = Dataset(
    train_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/anomaly/hazelnut/train",
    val_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/anomaly/hazelnut/test",
    test_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/anomaly/hazelnut/test",
)

In [3]:
train_dataloader = dataset.train_dataloader()
print(f"Dataset type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")

[*] Detected task type: ANOMALY_CLASSIFICATION
Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 28
Dataset size: 28


## Prepare Model
Config to build the model. -> Provide function for building models so that each framework's config can be used

    - Users can build a torch.nn.Module via config as well

In [4]:
model_config = {
    "model": {
        "name": "padim",
        "backbone": "resnet18",
        "pre_trained": True,
        "layers": ["layer1", "layer2", "layer3"],
        "normalization_method": "min_max",
        "input_size": [256, 256],
    }
}
model = get_model(model_config)
print(f"Model type: {type(model)}")

/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: DeprecationWarning: From v0.10 an `'Binary*'`, `'Multiclass*', `'Multilabel*'` version now exist of each classification metric. Moving forward we recommend using these versions. This base metric will still work as it did prior to v0.10 until v0.11. From v0.11 the `task` argument introduced in this metric will be required and the general order of arguments may change, such that this metric will just function as an single entrypoint to calling the three specialized versions.
  warnings.warn(*args, **kwargs)
/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Model type: <class 'anomalib.models.padim.lightning_model.PadimLightning'>


In [5]:
for i, data_batch in enumerate(train_dataloader):
    model.training_step(data_batch)

## Training

Users can use each framework's training provided by OTX. (Engine)

- The engine requires the necessary models and DataLoaders for each framework.

In [6]:
engine = AnomalibEngine()
work_path = "/tmp/OTX-API-test"
engine.train(
    model=model,
    train_dataloader=train_dataloader,
    work_dir=work_path,
    max_epochs=10,
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=2)`.
  rank_zero_warn(
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/pytorch_lightning/core/optimizer.py:183: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  ran

Epoch 0:  32%|███▏      | 9/28 [00:00<00:00, 54.28it/s, loss=nan, v_num=13]

/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:138: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Epoch 0:  64%|██████▍   | 18/28 [00:00<00:00, 56.62it/s, loss=nan, v_num=13]

Epoch 9: 100%|██████████| 28/28 [00:00<00:00, 56.36it/s, loss=nan, v_num=13]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 28/28 [00:00<00:00, 41.80it/s, loss=nan, v_num=13]


## OTX AutoEngine (Automation Training API)
OTX provides a more convenient API called AutoEngine.

- It's more convenient for users to use Engine, which provides auto-configuration and the features provided by OTX without having to choose a framework.
- Prepare Dataset & DataLoader + Prepare Model + OTX Recipes + Training + ETC.
- This will make all of the above steps happen automatically. (Auto: Model Selection & build, Dataset Configuration, Training, etc..)

In [7]:
from otx.v2.api.core.engine import AutoEngine

output_dir = "/tmp/OTX-API-test"
data_roots = "/home/harimkan/workspace/repo/otx-fork-3/tests/assets/anomaly/hazelnut/train"
default_config_path = "/home/harimkan/workspace/repo/otx-fork-3/src/otx/v2/configs/anomaly_classification/anomalib_padim.yaml"

engine = AutoEngine(
    task="anomaly_classification",
    work_dir=output_dir,
    train_data_roots=data_roots,
    config=default_config_path,
)

engine.train(batch_size=2, max_epochs=5)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.

  | Name            | Type                  | Params
----------------------------------------------------------
0 | image_threshold | AnomalyScoreThreshold | 0     
1 | pixel_threshold | AnomalyScoreThreshold | 0     
2 | model           | PadimModel            | 2.8 M 
----------------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.131    Total estimated model params size (MB)
/home/harimkan/workspace/repo/otx-fork-3/venv-origin/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (14) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you w

Epoch 0:   0%|          | 0/14 [00:00<?, ?it/s] 

Epoch 4: 100%|██████████| 14/14 [00:00<00:00, 30.65it/s, loss=nan, v_num=14]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 14/14 [00:00<00:00, 22.06it/s, loss=nan, v_num=14]
